<a href="https://colab.research.google.com/github/saadz-khan/Text2Image/blob/master/3D_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip3 install opencv-python==4.2.0.32
!pip3 install vispy==0.6.4
!pip3 install moviepy==1.0.2
!pip3 install transforms3d==0.3.1
!pip3 install networkx==2.3
!sudo apt install sed

%cd /content/
!git clone https://github.com/vt-vl-lab/3d-photo-inpainting.git
%cd 3d-photo-inpainting
!sh download.sh

!sed -i 's/offscreen_rendering: True/offscreen_rendering: False/g' argument.yml

%cd image
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
%cd ..

/content
Cloning into '3d-photo-inpainting'...
remote: Enumerating objects: 372, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 372 (delta 9), reused 15 (delta 9), pack-reused 350
Receiving objects: 100% (372/372), 130.14 MiB | 42.35 MiB/s, done.
Resolving deltas: 100% (201/201), done.
/content/3d-photo-inpainting
downloading from filebox ...
--2022-03-14 18:23:21--  https://filebox.ece.vt.edu/~jbhuang/project/3DPhoto/model/color-model.pth
Resolving filebox.ece.vt.edu (filebox.ece.vt.edu)... 128.173.88.43
Connecting to filebox.ece.vt.edu (filebox.ece.vt.edu)|128.173.88.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206331633 (197M)
Saving to: ‘color-model.pth’

color-model.pth     100%[===================>] 196.77M  34.0MB/s    in 6.4s    

2022-03-14 18:23:27 (30.8 MB/s) - ‘color-model.pth’ saved [206331633/206331633]

--2022-03-14 18:23:27--  https://filebox.ece.vt.edu/~jbhuang/project/3

- **Execute the 3D Photo Inpainting**
  - Note: The 3D photo generation process usually takes about 2-3 minutes or more depending on the available computing resources.

In [ ]:
from PIL import Image

im = Image.open("/content/3d-photo-inpainting/image/final3.jpg")
rgb_im = im.convert('RGB')
rgb_im.save('pixray.jpg')

In [12]:
%cd /content/3d-photo-inpainting/
!python main.py --config argument.yml

/content/3d-photo-inpainting
running on device 0
  0% 0/1 [00:00<?, ?it/s]Current Source ==>  final3
Running depth extraction at 1647283755.8506064
BoostingMonocularDepth/inputs/*.png
BoostingMonocularDepth/inputs/*.jpg
device: cuda
Namespace(Final=True, R0=False, R20=False, colorize_results=False, data_dir='inputs/', depthNet=0, max_res=inf, net_receptive_field_size=None, output_dir='outputs', output_resolution=1, pix2pixsize=1024, savepatchs=0, savewholeest=0)
----------------- Options ---------------
                    Final: True                          	[default: False]
                       R0: False                         
                      R20: False                         
             aspect_ratio: 1.0                           
               batch_size: 1                             
          checkpoints_dir: ./pix2pix/checkpoints         
         colorize_results: False                         
                crop_size: 672                           
          

- **The results are stored in the following directories**
  - Corresponding depth map estimated by [MiDaS](https://github.com/intel-isl/MiDaS.git) 
      - E.g. ```/content/3d-photo-inpainting/depth/moon.npy```
  - Inpainted 3D mesh
      - E.g. ```/content/3d-photo-inpainting/mesh/moon.ply```
  - Rendered videos with zoom-in motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_zoom-in.mp4```
  - Rendered videos with swing motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_swing.mp4```
  - Rendered videos with circle motion
      - E.g. ```/content/3d-photo-inpainting/video/moon_circle.mp4```
  
    ![](https://drive.google.com/uc?id=1xBiZAeGfhsoAsG08N5UM8-f993p0G5HF)

### GIF

In [14]:
import imageio
import os, sys

class TargetFormat(object):
    GIF = ".gif"
    MP4 = ".mp4"
    AVI = ".avi"

def convertFile(inputpath, targetFormat):
    """Reference: http://imageio.readthedocs.io/en/latest/examples.html#convert-a-movie"""
    outputpath = os.path.splitext(inputpath)[0] + targetFormat
    print("converting\r\n\t{0}\r\nto\r\n\t{1}".format(inputpath, outputpath))

    reader = imageio.get_reader(inputpath)
    fps = reader.get_meta_data()['fps']

    writer = imageio.get_writer(outputpath, fps=fps)
    for i,im in enumerate(reader):
        sys.stdout.write("\rframe {0}".format(i))
        sys.stdout.flush()
        writer.append_data(im)
    print("\r\nFinalizing...")
    writer.close()
    print("Done.")

convertFile("/content/3d-photo-inpainting/video/final3_dolly-zoom-in.mp4", TargetFormat.GIF)

converting
	/content/3d-photo-inpainting/video/final3_dolly-zoom-in.mp4
to
	/content/3d-photo-inpainting/video/final3_dolly-zoom-in.gif
frame 240
Finalizing...
Done.
